<a href="https://colab.research.google.com/github/deepiskp-99/DSA_Python_CaseStudy/blob/main/ETL_CaseStudy_Deepika.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import sqlite3

##**1. Load the dataset into a DataFrame**

In [22]:
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv "
df = pd.read_csv(url)

##**2.Identify number of records and unique locations**

In [23]:
num_records = len(df)
num_unique_places = df['place'].nunique()

print("Number of records:", num_records)
print("Unique places:", num_unique_places)

Number of records: 9887
Unique places: 5250


##**3.Print top 5 rows and column names**

In [24]:
print(df.head())


                       time   latitude   longitude  depth   mag magType   nst  \
0  2025-09-21T16:05:57.480Z  35.705667 -117.570333   9.12  0.76      ml  29.0   
1  2025-09-21T16:04:28.010Z  38.820835 -122.842163   2.29  0.97      md  15.0   
2  2025-09-21T15:45:33.898Z  59.256100 -153.104500  87.70  1.60      ml   NaN   
3  2025-09-21T15:44:09.233Z  60.541000 -144.903100  18.90  1.70      ml   NaN   
4  2025-09-21T15:38:47.453Z  52.886600  161.021300  10.00  4.70      mb  58.0   

     gap    dmin   rms  ...                   updated  \
0   43.0  0.1120  0.15  ...  2025-09-21T16:09:21.594Z   
1  103.0  0.0133  0.01  ...  2025-09-21T16:06:05.878Z   
2    NaN     NaN  0.28  ...  2025-09-21T15:47:15.575Z   
3    NaN     NaN  0.38  ...  2025-09-21T15:45:27.566Z   
4  165.0  1.4410  0.83  ...  2025-09-21T16:00:58.040Z   

                                          place        type horizontalError  \
0                    13 km NE of Ridgecrest, CA  earthquake            0.20   
1           

##**4.Convert time to datetime**

In [25]:
df['time'] = pd.to_datetime(df['time'])

##**5.Drop records with missing latitude, longitude, mag**

In [26]:
df = df.dropna(subset=['latitude', 'longitude', 'mag'])

##**6.Filter only earthquakes with magnitude >= 4.0**

In [27]:
df = df[df['mag'] >= 4.0]

##**7.Add a column day_of_week from time**

In [28]:
df['day_of_week'] = df['time'].dt.day_name()

##**8.● Create a column severity_level based on magnitude:**
**○ <4.0: "Low"**
**○ 4.0 - 6.0: "Moderate"**
**○ 6.0+: "High"**

In [29]:
def severity(mag):
    if mag < 4.0:
        return "Low"
    elif 4.0 <= mag < 6.0:
        return "Moderate"
    else:
        return "High"

df['severity_level'] = df['mag'].apply(severity)

##**9.Count number of earthquakes per place**

In [30]:
eq_counts = df['place'].value_counts().reset_index()
eq_counts.columns = ['place', 'earthquake_count']

##**10.Compute average magnitude and max depth per day**

In [31]:
summary = df.groupby(df['time'].dt.date).agg(
    avg_magnitude=('mag', 'mean'),
    max_depth=('depth', 'max')
).reset_index()

##**11.Save: ○ Cleaned dataset to cleaned_earthquakes.csv**
##**○ Summary data to earthquake_summary.csv**

In [32]:
df.to_csv("cleaned_earthquakes.csv", index=False)
summary.to_csv("earthquake_summary.csv", index=False)

##**12.Store into SQLite**

In [33]:
conn = sqlite3.connect("earthquakes.db")
df.to_sql("earthquakes", conn, if_exists="replace", index=False)
summary.to_sql("earthquake_summary", conn, if_exists="replace", index=False)
conn.close()

##**13.Create a flowchart of your ETL logic**

             ┌─────────────────────┐
             │   Extract Data      │
             │  (USGS CSV online)  │
             └─────────┬───────────┘
                       │
                       ▼
             ┌─────────────────────┐
             │   Transform Data    │
             │ - Convert time      │
             │ - Drop nulls        │
             │ - Filter mag >= 4   │
             │ - Add day_of_week   │
             │ - Add severity      │
             │ - Aggregations      │
             └─────────┬───────────┘
                       │
                       ▼
             ┌─────────────────────┐
             │     Load Data       │
             │ - Save CSV files    │
             │ - Save SQLite DB    │
             └─────────────────────┘


##**14.  A short explanation of your process**

Extract: Pulled earthquake data directly from the USGS monthly CSV feed.

Transform:

Converted timestamps into datetime.

Dropped incomplete records (latitude, longitude, mag missing).

Filtered earthquakes with mag ≥ 4.0.

Added day_of_week and severity_level.

Aggregate: Computed number of earthquakes per place, and daily summary with average magnitude and max depth.

Load: Stored results into CSV and SQLite for further analysis and sharing.

##**15.One insight your team discovered from the data**

Most earthquakes occur in the “Ring of Fire” regions (e.g., Alaska, Indonesia, Japan, Chile).
From initial inspection, moderate earthquakes (mag 4–6) dominate the dataset, while high severity (≥6) are rare but concentrated in seismic hotspots.